In [16]:
import sys
sys.path.append('..')
from spyral.core.constants import QBRHO_2_P
from spyral.core.cluster import Cluster
from spyral.solvers.solver_interp_leastsq import Guess, interpolate_trajectory
from spyral.interpolate.track_interpolator import create_interpolator
from e20009_phases.InterpLeastSqSolverPhase import fit_model_interp

from e20009_phases.config import  DetectorParameters
from spyral_utils.nuclear import NuclearDataMap

import polars as pl
import numpy as np
from pathlib import Path

import plotly.graph_objects as go
from plotly.subplots import make_subplots
%matplotlib widget

# from e20009_phases.InterpSolverPhase import solve_physics_interp


In [17]:
det_params = DetectorParameters(
    magnetic_field=3.0,
    electric_field=60000.0,
    detector_length=1000.0,
    beam_region_radius=20.0,
    drift_velocity_path=Path(
        "C:\\Users\\schaeffe\\Desktop\\e20009_analysis-dev_1_18_2025\\e20009_analysis-dev\\e20009_parameters\\drift_velocity.csv"
    ),
    get_frequency=3.125,
    garfield_file_path=Path(
        "/Users/attpc/Desktop/e20009_analysis/e20009_analysis/e20009_parameters/e20009_efield_correction.txt"
    ),
    do_garfield_correction=False,
)

In [18]:


run_number = 297
event_number = 590868

In [34]:
## Proton Cluster for event  ##

cluster_index = 0
cluster_data = np.loadtxt("C:\\Users\\schaeffe\\Desktop\\e20009_B11_output\\B10(d,p)\\14_5_16_MeV_4_track\\297_590868\\cluster0_proton.txt", delimiter=",")
cluster = Cluster(event_number, cluster_index, cluster_data)

## estimation_with_fake_input_wSQRTDEDX.ipynb ##

proton_polar_guess = 37.01420713 * np.pi/180.0   # Radians
proton_azimuthal_guess = 265.6313563  * np.pi/180.0   # Radians
proton_brho_guess = 0.7560472748  # Tm
proton_xvert_guess = -13.7578169  # mm
proton_yvert_guess = 1.05156147  # mm
proton_zvert_guess = 356.8465004   # mm


guess = Guess(proton_polar_guess, proton_azimuthal_guess, proton_brho_guess, proton_xvert_guess, proton_yvert_guess, proton_zvert_guess)
print(guess)

Guess(polar=0.6460197844336608, azimuthal=4.636141763973182, brho=0.7560472748, vertex_x=-13.7578169, vertex_y=1.05156147, vertex_z=356.8465004, direction=<Direction.NONE: -1>)


In [ ]:
## Triton Cluster ## 

# cluster_index = 3
# cluster_data = np.loadtxt("c:\\Users\\schaeffe\\Desktop\\e20009_B11_output\\B10(d,p)\\14_5_16_MeV_4_track\\297_590868\\cluster3_triton.txt", delimiter=",")
# cluster = Cluster(event_number, cluster_index, cluster_data)

# ## estimation_with_fake_input_wSQRTDEDX.ipynb ##

# triton_polar_guess = 17.88763272 * np.pi/180.0   # Radians
# triton_azimuthal_guess = 24.58734587 * np.pi/180.0   # Radians
# triton_brho_guess = 1.115307667   # Tm
# triton_xvert_guess = -0.6447775602  # mm
# triton_yvert_guess = 1.413200043  # mm
# triton_zvert_guess = 380.6705819  # mm

# guess = Guess(triton_polar_guess, triton_azimuthal_guess, triton_brho_guess, triton_xvert_guess, triton_yvert_guess, triton_zvert_guess)
# print(guess)

In [35]:
# Get drift velocity

dv_lf: pl.LazyFrame = pl.scan_csv(det_params.drift_velocity_path)
dv_df: pl.DataFrame = dv_lf.filter(pl.col("run") == run_number).collect()
mm_tb: float = dv_df.get_column("average_micromegas_tb")[0]
w_tb: float = dv_df.get_column("average_window_tb")[0]
mm_err: float = dv_df.get_column("average_micromegas_tb_error")[0]
w_err: float = dv_df.get_column("average_window_tb_error")[0]

# print(dv_df)

In [36]:
z = 1 # atomic number

a_p = 1
a_t = 3 # mass number
nuclear_map = NuclearDataMap()
proton = nuclear_map.get_data(z, a_p)
# triton = nuclear_map.get_data(z, a_t)
# print(nucleus.A)

In [37]:
path_to_ODE_solution_mesh = Path("C:\\Users\\schaeffe\\Desktop\\e20009_B11_output\\InterpSolver_assets\\1H_in_2H2_600Torr.npy")    # Mesh for protons 
# path_to_ODE_solution_mesh = Path("C:\\Users\\schaeffe\\Desktop\\e20009_B11_output\\InterpSolver_assets\\3H_in_2H2_600Torr.npy")    # Mesh for tritons
tracks = create_interpolator(path_to_ODE_solution_mesh)

In [38]:

print(tracks.polar_bins)

360


In [39]:
result = fit_model_interp(cluster, guess, proton, tracks, det_params, w_tb, mm_tb, w_err, mm_err)
if result is None:
    print('Guess outside of interpolation range!')
best_fit_trajectory = interpolate_trajectory(result, tracks, proton)
cluster.data[:, :3] *= 0.001



[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 447
    # data points      = 36
    # variables        = 6
    chi-square         = 11.3477255
    reduced chi-square = 0.37825752
    Akaike info crit   = -29.5620581
    Bayesian info crit = -20.0609445
[[Variables]]
    brho:        0.71039552 +/- 0.00990203 (1.39%) (init = 0.7560473)
    polar:       0.75429091 +/- 0.02332284 (3.09%) (init = 0.6460198)
    vertex_rho:  0.02000000 +/- 0.21176089 (1058.80%) (init = 0.01379795)
    vertex_phi:  3.21157479 +/- 0.27905081 (8.69%) (init = 3.065307)
    vertex_x:   -0.01995104 +/- 0.21116336 (1058.41%) == 'vertex_rho * cos(vertex_phi)'
    vertex_y:   -0.00139850 +/- 0.01684798 (1204.72%) == 'vertex_rho * sin(vertex_phi)'
    vertex_z:    0.39123998 +/- 0.00828787 (2.12%) (init = 0.3568465)
    azimuthal:   4.73009530 +/- 0.01933895 (0.41%) (init = 4.636142)
[[Correlations]] (unreported correlations are < 0.100)
    C(polar, vertex_z)        = +0.8867
    C(polar

In [40]:
proton_polar_final = result["polar"].value * 180 / np.pi
proton_azimuthal_final = result["azimuthal"].value
proton_brho_final = result["brho"].value
proton_xvert_final = result["vertex_x"].value
proton_yvert_final = result["vertex_y"].value
proton_zvert_final = result["vertex_z"].value

momentum = proton_brho_final  * QBRHO_2_P
kinetic_energy = np.sqrt(momentum**2.0 + proton.mass**2.0) - proton.mass
print(kinetic_energy)
print(proton_polar_final)

23.866806444079543
43.21768540983862


In [41]:
# triton_polar_final = result["polar"].value * 180 / np.pi
# triton_azimuthal_final = result["azimuthal"].value
# triton_brho_final = result["brho"].value
# triton_xvert_final = result["vertex_x"].value
# triton_yvert_final = result["vertex_y"].value
# triton_zvert_final = result["vertex_z"].value

# momentum = triton_brho_final  * QBRHO_2_P
# kinetic_energy = np.sqrt(momentum**2.0 + triton.mass**2.0) - triton.mass
# print(kinetic_energy)
# print(triton_polar_final)

In [33]:
# Plot fit
fig = make_subplots(2, 2, subplot_titles=["XY Projection", "XZ Projection", "YZ Projection"], specs=[[{"rowspan": 2}, {}],[None, {}]])
fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 0],
        y=cluster.data[:, 1],
        mode="markers", 
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 0],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit"
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_x"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex"
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 0],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False
    ),
    row=1,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_x"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False,
    ),
    row=1,
    col=2
)

fig.add_trace(
    go.Scatter(
        x=cluster.data[:, 2],
        y=cluster.data[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "blue"
        },
        name="Data",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=best_fit_trajectory[:, 2],
        y=best_fit_trajectory[:, 1],
        mode="markers",
        marker={
            "size": 3,
            "color": "red"
        },
        name="Fit",
        showlegend=False,
    ),
    row=2,
    col=2
)
fig.add_trace(
    go.Scatter(
        x=[result["vertex_z"]],
        y=[result["vertex_y"]],
        mode="markers",
        marker={
            "color": "green",
            "size": 4
        },
        name="Fit Vertex",
        showlegend=False
    ),
    row=2,
    col=2
)

fig["layout"]["xaxis1"]["title"] = "X (m)"
fig["layout"]["xaxis1"]["range"] = [-0.3, 0.3]
fig["layout"]["yaxis1"]["title"] = "Y (m)"
fig["layout"]["yaxis1"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis2"]["title"] = "Z (m)"
fig["layout"]["xaxis2"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis2"]["title"] = "X (m)"
fig["layout"]["yaxis2"]["range"] = [-0.3, 0.3]

fig["layout"]["xaxis3"]["title"] = "Z (m)"
fig["layout"]["xaxis3"]["range"] = [0.0, 1.0]
fig["layout"]["yaxis3"]["title"] = "Y (m)"
fig["layout"]["yaxis3"]["range"] = [-0.3, 0.3]

fig.update_layout(
    width=1450,
    height=725
)